In [1]:
import matplotlib as plt
import numpy as np
import pandas as pd
import json
import subprocess
from glob2 import glob

In [2]:
dcml_chordtype_map = {
    "M": "major",
    "m": "minor",
    "Mm7": "dominant-7th",
    "o": "diminished",
    "o7": "full-diminished",
    "mm7": "minor-7th",
    "%7": "half-diminished",
    "MM7": "major-7th",
    "+": "augmented",
    "mM7": "minor-major-7th",
    "+7": "augmented-7th",
}

----

In [16]:
slices = glob("inputs/slices/*.csv")


In [22]:
slices[0][14:-4]

'n15op132_03'

In [26]:
def runExp():
    slices = glob("inputs/slices/*.csv")
    pieceNames = [x[14:-4] for x in slices] # THis is cursed, please remove it at once >:(
    for piece in pieceNames:
        jsonPath = "outputs/"+piece+".json"
        cmd = ["stack","run","fullParse","--","-n","1",piece, "All"] 
        print("Running command: " + (" ".join(cmd)))
        print("Expecting results in " + jsonPath)
        #res = subprocess.run(cmd, cwd="..")
        res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")

        if res.returncode != 0:
            print("Error in subprocess")
            print(res.stderr)
            print(res.stdout)

In [ ]:
#runExp()

## Loading the Dataset
Loads all json files from the given folder

In [7]:
def load_dataset(path):
    jsonFiles = glob(path + '/*.json') #Can be used absolute or relative paths 
    df = pd.DataFrame() 
    for jsonFile in jsonFiles:
        # df = pd.read_json(jsonFile)
        try:
            with open(jsonFile, 'r') as f:
                data = json.loads(f.read())
        except:
            print("Error with {}".format(jsonFile))
            continue

        newdf = pd.json_normalize(data, record_path = ['results'], 
            meta = ['piece'])
        df = pd.concat([df, newdf])
    return df


In [8]:
x = load_dataset("outputs")

Error with outputs/n07op59-1_04.json
Error with outputs/n10op74_01.json
Error with outputs/n13op130_01.json
Error with outputs/n09op59-3_04.json
Error with outputs/n07op59-1_02.json
Error with outputs/n01op18-1_02.json
Error with outputs/n13op130_06.json
Error with outputs/n14op131_07.json
Error with outputs/n12op127_03.json
Error with outputs/n15op132_03.json
Error with outputs/n14op131_04.json
Error with outputs/n07op59-1_01.json
Error with outputs/n11op95_02.json
Error with outputs/n12op127_01.json
Error with outputs/n06op18-6_03.json
Error with outputs/n08op59-2_04.json
Error with outputs/n03op18-3_03.json
Error with outputs/shortt1.json


In [9]:
x

,accuracy,algorithm,chordLabels,likelihood,slices,piece
0,0.645161,RandomParseSBS,"[B♭M, FMm7, B♭M, B♭M, FMm7, B♭M, FMm7, B♭M, B♭...",-1.881735e+06,"[[D5, F5, B♭4, B♭4, B♭4, B♭3, G5], [E♭5, C4, A...",n06op18-6_01
1,0.413978,RandomSampleSBS,"[B♭M, FM, B♭M, FM, FM, B♭M, FMm7, B♭M, B♭M, GM...",-5.376495e+05,"[[D5, D5, B♭3], [F3, F5, A4, A4], [D5, F5], [D...",n06op18-6_01
2,0.029570,RandomSample,"[DMm7, DM, Cm, A♭M, A♭M, DMm7, A♭Mm7, CM, D♭Mm...",-1.075290e+06,"[[D3, C0, A4], [D3, E6, G♭-10, A4], [C0, C0, D...",n06op18-6_01
0,0.518248,RandomParseSBS,"[GM, DMm7, GM, DM, DM, DMm7, DM, GM, D♯o7, DMm...",-1.978066e+01,"[[D6, D5, B5, G6, G4, G5], [D6, D5, C♯6, C6, A...",n02op18-2_03
1,0.306569,RandomSampleSBS,"[GM, DMm7, AMm7, DM, DM, F♯o, DMm7, GM, CM, DM...",-1.846121e+01,"[[D5, G5], [D6, D6, C♯6, C6, F♯5], [C♯6, G4, G...",n02op18-2_03
...,...,...,...,...,...,...
1,0.332237,RandomSampleSBS,"[A♭M, Fm, D♭M, A♭M, D♭M, E♭Mm7, A♭M, A♭M, E♭M,...",-9.868590e+05,"[[E♭4, C5, A♭5, A♭3, A♭3], [D4, C7, C7, A♭3], ...",n10op74_02
2,0.023026,RandomSample,"[GM, DM, BM, F♭M, DM, FMm7, FMm7, B♭M, FM, E♭m...",-2.256926e+01,"[[D3, D3, G♭-10, G1, G1], [D3, C♯11, D♭-8, A4]...",n10op74_02
0,0.000000,RandomParseSBS,"[B♭m, G♭M, A♭M, D♭M, G♭M, E♭m, FM, B♭m, E♭m, A...",-4.129035e+07,"[[D♭5, F5, B♭3], [D♭5, B♭4, G♭5], [E♭5, C5, B♭...",n13op130_02
1,0.000000,RandomSampleSBS,"[B♭m, E♭m, E♭M, A♭M, G♭M, E♭M, FM, G♭M, E♭M, C...",-4.129034e+07,"[[D♭5, D♭5, F5, B♭4, B♭4], [E♭3, B♭4, G♭5], [E...",n13op130_02


In [12]:
x.groupby(["piece", "algorithm"]).agg([np.mean, np.std])

TypeError: mean() missing 1 required positional argument: 'a'